<div style="border: 3px solid #4CAF50; padding: 20px; font-size: 24px; font-weight: bold; font-family: 'Arial', sans-serif; text-align: center; line-height: 1.5; background: linear-gradient(to bottom, #e0f7e9, #ffffff); border-radius: 10px; box-shadow: 0 4px 8px rgba(0, 0, 0, 0.1);">
    מטלת סיכום חלק 2: למידת מכונה וכריית נתונים <br>
    :מגישי הפרוייקט <br> 
    יוסף בנקורמונו - 3184555904<br>
    אריאל קורן - 318284239 <br>
    <a href="https://github.com/arielk318/SemiProject/blob/main/semi%20final%20Project.ipynb">קישור לגיטהאב</a> <br>
</div>

In [1]:
import pandas as pd  # ייבוא pandas לעבודה עם מסגרות נתונים
import matplotlib.pyplot as plt  # ייבוא matplotlib ליצירת גרפים
import numpy as np  # ייבוא numpy לחישובים מתמטיים
import urllib.request, urllib.parse, urllib.error  # ייבוא ספריות לעבודה עם בקשות רשת
import requests  # ייבוא requests לשליחת בקשות HTTP
from sklearn.impute import KNNImputer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import f_regression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.linear_model import ElasticNet
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler, PolynomialFeatures
from sklearn.model_selection import GridSearchCV
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor
import seaborn as sns
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import IsolationForest
import optuna
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import cross_val_score, KFold
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score

In [3]:
df = pd.read_csv("dataset.csv")
# df

In [4]:
na_per_column = df.isnull().sum(axis=0)
na_per_column

manufactor            0
Year                  0
model                 0
Hand                  0
Gear                  1
capacity_Engine      26
Engine_type           5
Prev_ownership      729
Curr_ownership      728
Area                149
City                  0
Price                 0
Pic_num              24
Cre_date              0
Repub_date            0
Description           0
Color               770
Km                   89
Test               1368
Supply_score       1061
dtype: int64

In [5]:
# Function to convert string numbers with commas to float
def convert_to_float(value):
    if value is None or value == 'None':
        return float('nan')
    if isinstance(value, str):
        return float(value.replace(',', ''))
    return value

In [6]:
# Function to replace low values with the mean of the same manufactor and model
def replace_with_mean(group):
    threshold = 600
    # Calculate mean excluding 0 and values below threshold
    mean_value = group[(group['capacity_Engine'] >= threshold)]['capacity_Engine'].mean()
    # Replace values below threshold with the mean, ignoring 0
    group.loc[(group['capacity_Engine'] < threshold) & (group['capacity_Engine'] != 0), 'capacity_Engine'] = mean_value
    return group

In [7]:
## This function imputes missing values using KNN for several columns:
##  ['Engine_type','Year','Hand','Km','Pic_num','capacity_Engine']
def Fill_Engine_type(df):
    # Initialize LabelEncoder for categorical columns
    label_encoder = LabelEncoder()
    
    # Encode categorical columns to numerical values
    df['Engine_type_encoded'] = label_encoder.fit_transform(df['Engine_type'].astype(str))
    
    # Columns to impute
    impute_columns = ['Year','Hand','Km','Pic_num','capacity_Engine', 'Engine_type_encoded']
    
    # Create KNNImputer object
    imputer = KNNImputer(n_neighbors=2)
    
    # Impute missing values
    df[impute_columns] = imputer.fit_transform(df[impute_columns])
    
    # Convert encoded values back to categorical
    df['Engine_type'] = label_encoder.inverse_transform(df['Engine_type_encoded'].astype(int))
    
    # Drop the temporary encoded columns
    df.drop(columns=['Engine_type_encoded'], inplace=True)
    
    return df

In [8]:
# This function impute missing values after encoded columns : manufactor,gear,model.
def Fill_Gear(df):
    # Encode 'manufactor' and 'model' columns
    label_encoder_manufactor = LabelEncoder()
    df['manufactor_encoded'] = label_encoder_manufactor.fit_transform(df['manufactor'].astype(str))

    label_encoder_model = LabelEncoder()
    df['model_encoded'] = label_encoder_model.fit_transform(df['model'].astype(str))

    # Encode 'Gear' column
    label_encoder_gear = LabelEncoder()
    df['Gear_encoded'] = label_encoder_gear.fit_transform(df['Gear'].astype(str))

    # Create KNNImputer object
    imputer = KNNImputer(n_neighbors=2)

    # Impute missing values for encoded columns
    impute_columns = ['manufactor_encoded', 'model_encoded', 'Gear_encoded']
    df[impute_columns] = imputer.fit_transform(df[impute_columns])

    # Convert encoded values back to categorical
    df['manufactor'] = label_encoder_manufactor.inverse_transform(df['manufactor_encoded'].astype(int))
    df['model'] = label_encoder_model.inverse_transform(df['model_encoded'].astype(int))
    df['Gear'] = label_encoder_gear.inverse_transform(df['Gear_encoded'].astype(int))

    # Drop the temporary encoded columns
    df.drop(columns=['manufactor_encoded', 'model_encoded', 'Gear_encoded'], inplace=True)

    return df

In [9]:
def preprocess_data(df):
    # Define categorical columns for OneHotEncoder
    categorical_columns = ['Gear', 'model', 'manufactor', 'Engine_type']
    
    # Drop non-numeric columns that are not needed for OneHotEncoder
    df_numeric = df.drop(categorical_columns, axis=1)
    
    # One-hot encode categorical columns
    encoder = OneHotEncoder(sparse=False, handle_unknown='ignore', drop='first')
    encoded_cols = pd.DataFrame(encoder.fit_transform(df[categorical_columns]), columns=encoder.get_feature_names_out(categorical_columns))
    
    # Concatenate numeric and encoded categorical columns
    df = pd.concat([df_numeric, encoded_cols], axis=1)
    
    return df

In [10]:
def prepare_data(df):
    
    # Drop rows with NA in 'Price'
    df = df.dropna(subset=['Price'])
    # remove columns that not neccesery 
    columns_to_drop = ['Prev_ownership','Curr_ownership','Color','Area','City', 'Cre_date', 'Repub_date', 'Description', 'Supply_score','Test']
    df = df.drop(columns=columns_to_drop)
    
    # Convert 'Year', 'Km','capacity_Engine' and 'Pic_num' to numeric
    df['Year'] = pd.to_numeric(df['Year'], errors='coerce')
    df['Km'] = df['Km'].apply(convert_to_float)
    df['capacity_Engine'] = df['capacity_Engine'].apply(convert_to_float)
    df['Pic_num'] = df['Pic_num'].apply(convert_to_float)
    
    # If Pic_num is NA fill with 0
    df['Pic_num'].fillna(0, inplace=True)
    # Correcting values of Km column: multiplying values less than 1000 by 1000
    df['Km'] = df['Km'].apply(lambda x: x*1000 if x < 1000 else x)
    
    # Replace special values
    df['Gear'] = df['Gear'].replace({'אוטומט': 'אוטומטית', 'לא מוגדר': np.nan})
    # These scripts will replace all instances of "היבריד" with "היברידי" in the Engine_type column  
    df['Engine_type'] = df['Engine_type'].replace('היבריד', 'היברידי')
    # These scripts will replace all instances of "Lexsus" with "לקסוס" in the manufactor column  
    df['manufactor'] = df['manufactor'].replace('לקסוס','Lexsus')
    # Change the Engine type of the electric car to zero     
    df.loc[df['Engine_type'] == 'חשמלי', 'capacity_Engine'] = 0
    
    # Calculate mean Km by Year
    mean_km_by_year = df.groupby('Year')['Km'].mean()
    # Replace zero Km values with mean values based on Year
    df['Km'] = df.apply(lambda row: mean_km_by_year[row['Year']] if row['Km'] == 0 else row['Km'], axis=1)
    
    # Filter rows where capacity_Engine > 10000 and divide those values by 10 outliers
    df.loc[df['capacity_Engine'] > 10000, 'capacity_Engine'] /= 10
    
    # Apply the function to fillNA at catagorical col(Engine_type) and numerical columns . 
    df = Fill_Engine_type(df)
    
    # Impute missing values after making catagorical columns encoded:
    # ['manufactor','gear','model']
    df = Fill_Gear(df)
    
    ### one hot encoder for catagorical columns: 
    # Apply preprocess_data func on our data- to make One Hot Encoder
    df = preprocess_data(df)
     
    # Define a threshold for low engine capacity values
    threshold = 600

    # Find all columns related to manufactor and model
    manufactor_columns = [col for col in df.columns if col.startswith('manufactor_')]
    model_columns = [col for col in df.columns if col.startswith('model_')]

    # Combine manufactor and model columns to group by
    group_columns = manufactor_columns + model_columns

    # Group by manufactor and model columns and apply the replacement function
    df = df.groupby(group_columns).apply(replace_with_mean).reset_index(drop=True)

    return df


In [11]:
prepare_data(df)

C:\Users\ariel\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


,Year,Hand,capacity_Engine,Price,Pic_num,Km,Gear_אוטומטית,Gear_טיפטרוניק,Gear_ידנית,Gear_רובוטית,...,manufactor_קיה,manufactor_קרייזלר,manufactor_רנו,manufactor_שברולט,Engine_type_בנזין,Engine_type_גז,Engine_type_דיזל,Engine_type_היברידי,Engine_type_חשמלי,Engine_type_טורבו דיזל
0,2008.0,4.0,3200.0,60000.0,4.0,180000.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,2008.0,4.0,2500.0,60000.0,6.0,218000.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2,2014.0,3.0,2500.0,82000.0,3.0,219000.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,2014.0,3.0,2150.0,82000.0,4.0,163000.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,2014.0,3.0,1746.5,73000.0,1.0,202950.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1495,2007.0,2.0,2000.0,21000.0,1.0,150000.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1496,2010.0,5.0,1800.0,42000.0,7.0,159000.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1497,2006.0,4.0,2000.0,38000.0,8.0,227000.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1498,2016.0,2.0,1400.0,93000.0,9.0,110000.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [12]:
df = prepare_data(df)
# nan_rows = df[df['Km'].isna()]
# nan_rows
# df['capacity_Engine'].unique()
# missing_values = df.isna().sum().sum()
# print(f"Total missing values: {missing_values}")

C:\Users\ariel\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [13]:
na_per_column = df.isnull().sum(axis=0)
na_per_column
 

Year                      0
Hand                      0
capacity_Engine           0
Price                     0
Pic_num                   0
                         ..
Engine_type_גז            0
Engine_type_דיזל          0
Engine_type_היברידי       0
Engine_type_חשמלי         0
Engine_type_טורבו דיזל    0
Length: 329, dtype: int64

In [14]:
# Assuming df_encoded is your DataFrame containing the dataset
# And you want to calculate VIF for all columns except the dependent variable

# Select only numerical columns for VIF calculation
X = df.select_dtypes(include=['number'])

# Add a constant to the model (required by statsmodels for calculating VIF)
X = sm.add_constant(X)

# Calculate VIF for each feature
vif_data = pd.DataFrame()
vif_data["Feature"] = X.columns
vif_data["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]

# Print the VIF values
print(vif_data)

C:\Users\ariel\anaconda3\Lib\site-packages\statsmodels\regression\linear_model.py:1781: RuntimeWarning: divide by zero encountered in scalar divide
  return 1 - self.ssr/self.centered_tss
C:\Users\ariel\anaconda3\Lib\site-packages\statsmodels\stats\outliers_influence.py:198: RuntimeWarning: divide by zero encountered in scalar divide
  vif = 1. / (1. - r_squared_i)


                    Feature        VIF
0                     const   0.000000
1                      Year   9.263204
2                      Hand   2.434198
3           capacity_Engine   9.338758
4                     Price   6.659879
..                      ...        ...
325          Engine_type_גז  18.660589
326        Engine_type_דיזל  25.587624
327     Engine_type_היברידי  35.342000
328       Engine_type_חשמלי   5.060306
329  Engine_type_טורבו דיזל   5.886251

[330 rows x 2 columns]


In [15]:
# Drop features with high VIF
threshold = 5
# threshold = 10
features_to_drop = vif_data[vif_data['VIF'] > threshold]['Feature']
df_reduced = df.drop(columns=features_to_drop)
print(df_reduced)

      Hand  Pic_num        Km  model_ A3  model_ A4  model_ A5  model_ A6  \
0      4.0      4.0  180000.0        0.0        0.0        0.0        0.0   
1      4.0      6.0  218000.0        0.0        0.0        0.0        0.0   
2      3.0      3.0  219000.0        0.0        0.0        0.0        0.0   
3      3.0      4.0  163000.0        0.0        0.0        0.0        0.0   
4      3.0      1.0  202950.0        0.0        0.0        0.0        0.0   
...    ...      ...       ...        ...        ...        ...        ...   
1495   2.0      1.0  150000.0        1.0        0.0        0.0        0.0   
1496   5.0      7.0  159000.0        1.0        0.0        0.0        0.0   
1497   4.0      8.0  227000.0        1.0        0.0        0.0        0.0   
1498   2.0      9.0  110000.0        1.0        0.0        0.0        0.0   
1499   3.0      4.0  118000.0        1.0        0.0        0.0        0.0   

      model_ Q3  model_ R8  model_ RS5  ...  model_אאודי A3  model_אאודי A4

<h1>Machine learning model</h1>

In [16]:
X = df.drop(columns = "Price")
y = df['Price']

# Remove any remaining rows with NaN values
X = X.dropna()
y = y[X.index]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
# Scale the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)
# Define the model
# הגדרת הטווחים של הפרמטרים
param_distributions = {
    'alpha': np.logspace(-3, -1, 10),  # טווח של alpha בין 0.001 ל-0.1
    'l1_ratio': np.linspace(0.0, 1.0, 10)  # טווח של l1_ratio בין 0 ל-1
}

# יצירת מודל ElasticNet
model = ElasticNet()

# הגדרת RandomizedSearchCV
random_search = RandomizedSearchCV(model, param_distributions, n_iter=5, scoring='neg_mean_squared_error', cv=5, random_state=42)

# אימון RandomizedSearchCV על קבוצת האימון
random_search.fit(X_train_scaled, y_train)

# פרמטרים אופטימליים מ-RandomizedSearchCV
best_params = random_search.best_params_
best_alpha = best_params['alpha']
best_l1_ratio = best_params['l1_ratio']

# אימון המודל עם הפרמטרים האופטימליים
best_elastic_net_model = ElasticNet(alpha=best_alpha, l1_ratio=best_l1_ratio)
best_elastic_net_model.fit(X_train_scaled, y_train)

# חיזוי על קבוצת הבדיקה
y_pred = best_elastic_net_model.predict(X_test_scaled)

# חישוב שגיאת ממוצע הריבועים (MSE)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
rmse = np.sqrt(mse)

print(f"Best Parameters: {best_params}")
print(f"Mean Squared Error: {mse}")
print(f'RMSE: {rmse}')
print(f"R-squared Score: {r2}")


C:\Users\ariel\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 4.331e+10, tolerance: 4.547e+07 Linear regression models with null weight for the l1 regularization term are more efficiently fitted using one of the solvers implemented in sklearn.linear_model.Ridge/RidgeCV instead.
  model = cd_fast.enet_coordinate_descent(
C:\Users\ariel\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 4.225e+10, tolerance: 4.646e+07 Linear regression models with null weight for the l1 regularization term are more efficiently fitted using one of the solvers implemented in sklearn.linea

Best Parameters: {'l1_ratio': 0.3333333333333333, 'alpha': 0.05994842503189409}
Mean Squared Error: 131753666.8941109
RMSE: 11478.400014553898
R-squared Score: 0.7289147901687849


C:\Users\ariel\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.132e+10, tolerance: 4.578e+07
  model = cd_fast.enet_coordinate_descent(


<h1> Elastic Net model


In [21]:
#This code uses Optuna to perform hyperparameter optimization for an ElasticNet model by conducting 100 trials with different hyperparameter values. 
#It trains the model with the best hyperparameters, evaluates its performance using various performance metrics, 
#and performs 10-fold cross-validation to assess the stability of the performance.


<h4 style="color:black;font-weight:bold;"> Perform 10-fold cross-validation and report the performance metrics</h4>


In [22]:
#10-fold cross-validation is a technique where the dataset is divided into 10 equal parts (folds). 
#The model is trained on 9 folds and tested on the remaining fold, and this process is repeated 10 times, with each fold serving as the test set once. 
#The results are averaged to provide a robust estimate of the model's performance.

#Positive and Negative R-squared Values:
#Positive R-squared values indicate that the model explains the variability in the data well.
#Negative R-squared values indicate that the model performs worse than a simple average guess (i.e., the mean of the target values).

In [23]:
# # Perform 10-fold cross-validation and report the performance metrics
cv_scores = cross_val_score(best_elastic_net_model, scaler.transform(X), y, cv=10, scoring='r2')
print(f'10-fold cross-validation R-squared scores: {cv_scores}')
print(f'Mean R-squared score: {cv_scores.mean()}')

10-fold cross-validation R-squared scores: [-0.21044944  0.44985905  0.43029852  0.32952789  0.107322   -0.20790459
  0.7594075   0.68291908  0.78185786 -0.30717159]
Mean R-squared score: 0.28156662766421475


In [24]:
# Identify the five most impactful features
coefficients = pd.DataFrame({'Feature': X.columns, 'Coefficient': best_elastic_net_model.coef_})
coefficients['Absolute_Coefficient'] = coefficients['Coefficient'].abs()
top_features = coefficients.sort_values(by='Absolute_Coefficient', ascending=False).head(5)
print("Top 5 features with the highest impact:")
print(top_features[['Feature', 'Coefficient']])

Top 5 features with the highest impact:
                Feature   Coefficient
0                  Year  16458.849336
161  model_לנסר ספורטבק  -4118.348167
4                    Km  -4098.636296
44            model_325   3814.837097
2       capacity_Engine   3657.755726
